# coroutine

- coroutine은 채ㅐperative routine의 약자로 일반적으로 알고 있는 함수나 메서드 같은 서브루틴이 메인루틴과 종속관계를 가진 것과 다르게, 메인루틴과 대등한 관계로 협력하는 모습에서 코루틴이라고 불리게 되었다.

In [1]:
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
        
    except GeneratorExit:
        db_handler.close()

- sned() 전에 next()를 먼저 호출해야 한다는 것을 꼭 기억하자.(그렇지 않으면 TypeError가 발생한다.)
- next()를 반드시 호출해야하지만 그렇지 않기 위해서 @prepare_coroutine이라는 데코레이터를 사용하면 편리하게 사용할 수 있다.

In [ ]:
@prepare_coroutine
def stream_db_records(db_handler):
    retrieved_data = None
    page_size = 10
    try:
        while True:
            page_size = (yield retrieved_data) or page_size
            retrieved_data = db_handler.read_n_records(page_size)
        
    except GeneratorExit:
        db_handler.close()

---

- 제너레이터를 사용해 두 개의 값을 생성하고 세 번째 값을 반환한다. 마지막 리턴되는 값을 구하기 위해 예외를 처리하는 방법과 예외에서 어떻게 값을 구하는지를 알아두자

In [3]:
def generator():
    yield 1
    yield 2
    return 3

In [4]:
value = generator()

In [5]:
next(value)

1

In [6]:
next(value)

2

In [7]:
try:
    next(value)
except StopIteration as e:
    print(">>>>>>>>>returned value ", e.value)

>>>>>>>>>returned value  3


## yield form 사용 예

In [10]:
def chain1(*iterables):
    for it in iterables:
        for value in it:
            yield value

In [11]:
'''
yield form 구문을 사용하면 서브 제너레이터에서 직접 값을 생산할 수 있으므로 중첩 루프를 피할 수 있다. 
위 코드를 yield from 구문을 사용해 다음과 같이 코드를 단순화할 수 있다.
'''

def chain2(*iterables):
    for it in iterables:
        yield from it

In [12]:
dummy = ["test", "test2", "asdfas"], ("a", "b"), "caed"

In [15]:
list(chain1(dummy))

[['test', 'test2', 'asdfas'], ('a', 'b'), 'caed']

In [16]:
list(chain2(dummy))

[['test', 'test2', 'asdfas'], ('a', 'b'), 'caed']

## 서브 제너레이터에서 반환된 값 구하기

In [27]:
def sequence(name, start, end):
    print(f"{name} 제너레이터 {start}에서 시작")
    yield from range(start, end)
    print(f"{name} 제너레이터 {end}에서 종료")
    return end

In [28]:
def main():
    step1 = yield from sequence("first", 0, 5)
    step2 = yield from sequence("second", step1, 10)
    return step1 + step2

In [29]:
g = main()

In [30]:
next(g)

first 제너레이터 0에서 시작


0

In [31]:
next(g)

1

In [32]:
next(g)

2

In [33]:
next(g)

3

In [34]:
next(g)

4

In [35]:
next(g)

first 제너레이터 5에서 종료
second 제너레이터 5에서 시작


5

In [36]:
next(g)

6

In [37]:
next(g)

7

In [38]:
next(g)

8

In [39]:
next(g)

9

In [40]:
next(g)

second 제너레이터 10에서 종료


StopIteration: 15